# GradCAM Visualization Demo with ResNet101

Requirement:

* GPU Memory: 6GB or higher


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import tensorflow as tf
import numpy as np

from nets import resnet_v1
slim = tf.contrib.slim

import utils

# Create mini-batch for demo

img1 = utils.load_image("./demo.png", normalize=False)
img2 = utils.load_image("./shihtzu_mypuppy.jpg", normalize=False)
img3 = utils.load_image("./tiger.jpg", normalize=False)

batch1_img = img1.reshape((1, 224, 224, 3))
batch1_label = np.array([1 if i == 242 else 0 for i in range(1000)])  # 1-hot result for Boxer
batch1_label = batch1_label.reshape(1, -1)

batch2_img = img2.reshape((1, 224, 224, 3))
batch2_label = np.array([1 if i == 155 else 0 for i in range(1000)])  # 1-hot result for Shih-Tzu
batch2_label = batch2_label.reshape(1, -1)

batch3_img = img3.reshape((1, 224, 224, 3))
batch3_label = np.array([1 if i == 292 else 0 for i in range(1000)])  # 1-hot result for tiger
batch3_label = batch3_label.reshape(1, -1)

batch_img = np.concatenate((batch1_img, batch2_img, batch3_img), 0)
batch_label = np.concatenate((batch1_label, batch2_label, batch3_label), 0)
batch_size = 3

# batch_img = np.concatenate((batch1_img), 0)
# batch_label = np.concatenate((batch1_label), 0)
# batch_size = 1
# batch_label = batch_label.reshape(batch_size, -1)

# Create tensorflow graph for evaluation
eval_graph = tf.Graph()
with eval_graph.as_default():

    images = tf.placeholder("float", [batch_size, 224, 224, 3])
    labels = tf.placeholder(tf.float32, [batch_size, 1000])

    preprocessed_images = utils.resnet_preprocess(images)

    with slim.arg_scope(resnet_v1.resnet_arg_scope()):
        with slim.arg_scope([slim.batch_norm], is_training=False):
            # is_training=False means batch-norm is not in training mode. Fixing batch norm layer.
            # net is logit for resnet_v1. See is_training messing up issue: https://github.com/tensorflow/tensorflow/issues/4887
            net, end_points = resnet_v1.resnet_v1_101(preprocessed_images, 1000)
    prob = end_points['predictions'] # after softmax


    init = tf.global_variables_initializer()

    # Load resnet v1 weights

    # latest_checkpoint = tf.train.latest_checkpoint("model/resnet_v1_50.ckpt")
    latest_checkpoint = "model/resnet_v1_101.ckpt"
    ## Optimistic restore.
    reader = tf.train.NewCheckpointReader(latest_checkpoint)
    saved_shapes = reader.get_variable_to_shape_map()
    variables_to_restore = tf.global_variables()
    for var in variables_to_restore:
        if not var.name.split(':')[0] in saved_shapes:
            print("WARNING. Saved weight not exists in checkpoint. Init var:", var.name)
        else:
        # print("Load saved weight:", var.name)
            pass

    var_names = sorted([(var.name, var.name.split(':')[0]) for var in variables_to_restore
            if var.name.split(':')[0] in saved_shapes])
    restore_vars = []
    with tf.variable_scope('', reuse=True):
        for var_name, saved_var_name in var_names:
            try:
                curr_var = tf.get_variable(saved_var_name)
                var_shape = curr_var.get_shape().as_list()
                if var_shape == saved_shapes[saved_var_name]:
                    # print("restore var:", saved_var_name)
                    restore_vars.append(curr_var)
            except ValueError:
                print("Ignore due to ValueError on getting var:", saved_var_name) 
    saver = tf.train.Saver(restore_vars)

        
        
# Run tensorflow 

with tf.Session(graph=eval_graph) as sess:    
    sess.run(init)    
    # sess.run(tf.local_variables_initializer())
    saver.restore(sess, latest_checkpoint)
    
    prob = sess.run(prob, feed_dict={images: batch_img})
    
    print prob

/home/lxq/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


/home/lxq/anaconda2/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Restoring parameters from model/resnet_v1_101.ckpt
[[1.7806985e-04 1.4510260e-04 2.5851466e-04 ... 1.8010206e-04
  5.3205260e-04 5.7030929e-04]
 [6.7657452e-06 4.7765738e-05 2.3799188e-05 ... 3.6346453e-06
  1.6410889e-04 1.6715719e-03]
 [3.1226675e-04 1.9335221e-04 1.0050055e-04 ... 4.9374643e-04
  2.6269518e-03 7.0340105e-04]]
